In [1]:
from sentence_transformers import SentenceTransformer
from docuverse.utils import open_stream, read_config_file
import pickle
from tqdm.autonotebook import tqdm, trange
from docuverse.utils.text_tiler import TextTiler
from docuverse.engines.search_data import SearchData
from docuverse.engines.data_template import DataTemplate
import os

In [ ]:
# model_name = "thenlper/gte-small"
# model_name = "/home/raduf/mnlp-models-rag/06302024/slate.30m.english.rtrvr"
model_name = "ibm-granite/granite-embedding-30m-english"
model = SentenceTransformer(model_name)
max_size = 400
stride = 100
tiler = TextTiler(max_doc_size=max_size, stride=stride, tokenizer=model.tokenizer, aligned_on_sentences=False, text_trim_to=400)

ERROR! Session/line number was not unique in database. History logging moved to new session 21


Traceback (most recent call last):
  File "/home/raduf/.local/share/JetBrains/Toolbox/apps/pycharm-professional/plugins/python-ce/helpers/pydev/_pydevd_bundle/pydevd_comm.py", line 744, in make_thread_stack_str
    if is_real_file(my_file) and not is_jup_cell:
       ^^^^^^^^^^^^^^^^^^^^^
  File "/home/raduf/.local/share/JetBrains/Toolbox/apps/pycharm-professional/plugins/python-ce/helpers/pydev/pydevd_file_utils.py", line 559, in is_real_file
    return not _is_int(filename) and not filename.startswith("<ipython-input")
                                         ^^^^^^^^^^^^^^^^^^^
AttributeError: 'tuple' object has no attribute 'startswith'


In [ ]:

# datadir="data/clapnq_small"
datadir="data/long_example"

In [ ]:
file = "Long programmatic PDF.md"
if file == "passages.tsv":
     import pandas as pd
     data = pd.read_csv(os.path.join(datadir, "passages.tsv"), sep="\t")
     sents = data['text']
elif file == "Long programmatic PDF.md":
    sents = ["".join(open_stream(os.path.join(datadir, "Long programmatic PDF.md")).readlines())]

In [ ]:
sents[0][:1000]

In [ ]:
tiler.tokenizer.cls_token

In [ ]:
tiled_data = [tiler.create_tiles(i, txt, '', max_doc_size=512, stride=100, title_handling="none") for i, txt in enumerate(sents)]

In [ ]:
tiled_data[0][:10]

# Run the tiler in parallel

In [ ]:
from docuverse.utils import parallel_process

def proc_text(item):
     res = tiler.create_tiles(item['id'], item['text'], item['title'])
     item['result'] = res
     return item

res = parallel_process(proc_text, [{'id': i, 'text': txt, 'title': f"Title for item {i}"} for i, txt in enumerate(sents[:100])], num_threads=10)

In [ ]:
res